# Evaluation of our predictions

## Imports

In [1]:
import os
import time

import cv2
import numpy as np
import torch
from torch.utils.data.sampler import SequentialSampler
from torch_geometric.data import DataLoader

from pg_networks.gcn import GCN
from src.davis_2016 import DAVIS2016
from src.metrics import db_eval_iou, db_eval_boundary, db_eval_t_stab
from src.vis_utils import compute_combo_img

# for auto-reloading extenrnal modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

np.set_printoptions(precision=3)

## Choose model

In [2]:
model_path = 'pg_models/trained_model.pth'

## Paths & Constants

In [3]:
# Paths
PYTORCH_GEOMETRIC_DAVIS_2016_DATASET_PATH = 'pg_datasets/DAVIS_2016'
ANNOTATIONS_FOLDERS_PATH = 'DAVIS_2016/DAVIS/Annotations/480p/'
CONTOURS_FOLDERS_PATH = 'DAVIS_2016/DAVIS/Contours/480p'
IMAGES_FOLDERS_PATH = 'DAVIS_2016/DAVIS/JPEGImages/480p'
TRANSLATIONS_FOLDERS_PATH = 'DAVIS_2016/DAVIS/Translations/480p'

OSVOS_RESULTS_FOLDERS_PATH = 'OSVOS_PyTorch/models/Results/'

COMBO_RESULTS_FOLDERS_PATH = 'evaluations/'

# Sequences
SKIP_SEQUENCES = []

#SKIP_SEQUENCES = ['bmx-trees', 'bus', 'cows', 'dog-agility', 'horsejump-high', 
#                 'horsejump-low', 'kite-walk', 'lucia', 'libby', 'motorbike',
#                 'paragliding', 'rhino', 'scooter-gray', 'swing']

TRAIN_SEQUENCES = ['bear', 'bmx-bumps', 'boat', 'breakdance-flare', 'bus', 
                   'car-turn', 'dance-jump', 'dog-agility', 'drift-turn', 
                   'elephant', 'flamingo', 'hike', 'hockey', 'horsejump-low', 
                   'kite-walk', 'lucia', 'mallard-fly', 'mallard-water', 
                   'motocross-bumps', 'motorbike', 'paragliding', 'rhino', 
                   'rollerblade', 'scooter-gray', 'soccerball', 'stroller',
                   'surf', 'swing', 'tennis', 'train']

VAL_SEQUENCES = ['blackswan', 'bmx-trees', 'breakdance', 'camel', 'car-roundabout',
                 'car-shadow', 'cows', 'dance-twirl', 'dog', 'drift-chicane', 
                 'drift-straight', 'goat', 'horsejump-high', 'kite-surf', 'libby', 
                 'motocross-jump', 'paragliding-launch', 'parkour', 'scooter-black', 
                 'soapbox']

# Dataset parameters
EPOCHS_WO_AVEGRAD = 200
LAYER = 9
K = 32
NUM_SEQUENCES = 10

## Eval Dataset

In [4]:
val = DAVIS2016(PYTORCH_GEOMETRIC_DAVIS_2016_DATASET_PATH, 
                CONTOURS_FOLDERS_PATH, IMAGES_FOLDERS_PATH, TRANSLATIONS_FOLDERS_PATH, 
                LAYER, K, EPOCHS_WO_AVEGRAD,
                SKIP_SEQUENCES, TRAIN_SEQUENCES[:NUM_SEQUENCES], VAL_SEQUENCES[:NUM_SEQUENCES],
                train=False)

In [5]:
print("Val size: %i" % len(val))

Val size: 715


## Load GCN Model

In [6]:
model = GCN(in_channels=val[0].num_features, out_channels=val[0].y.shape[1])
model.load_state_dict(torch.load(model_path))
model.eval()
model.double()

GCN(
  (conv1): GCNConv(256, 512)
  (conv2): GCNConv(512, 512)
  (conv3): GCNConv(512, 1024)
  (lin1): Linear(in_features=1024, out_features=512, bias=True)
  (lin2): Linear(in_features=512, out_features=256, bias=True)
  (lin3): Linear(in_features=256, out_features=2, bias=True)
)

## Evaluate Model

In [7]:
val_loader = DataLoader(val, batch_size=1, shuffle=False, sampler=SequentialSampler(val))
file_names = val.processed_file_names

for i, data in enumerate(val_loader):
    
    #if i > 0: break
    
    # Forward pass to get outputs
    with torch.no_grad():
        translation_0_1_pred = model(data)
        
    # Get folder and filename for images
    try:
        file_name_1 = file_names[i + 1]    
    # Break if file_name_1 is last frame
    except IndexError as e:
        mean_J_combo = running_J_combo / running_index
        mean_J_osvos = running_J_osvos / running_index
        mean_F_combo = running_F_combo / running_index
        mean_F_osvos = running_F_osvos / running_index
        
        print(folder_old)
        print('\tmean_J_combo: {}, mean_J_osvos: {}'.format(mean_J_combo, mean_J_osvos))
        print('\tmean_F_combo: {}, mean_F_osvos: {}'.format(mean_F_combo, mean_F_osvos))
        break
        
    folder_1 = file_name_1[:-9]    
    file_name_1 = file_name_1[-8:-3]
    
    # Load OSVOS result image
    osvos_img_1_path = os.path.join(OSVOS_RESULTS_FOLDERS_PATH, folder_1, 
                                    ('{}{}'.format(file_name_1, '.png')))
    osvos_img_1 = cv2.imread(osvos_img_1_path)
    osvos_img_1_gray = cv2.imread(osvos_img_1_path, cv2.IMREAD_GRAYSCALE)
    
    # Load ground truth annotation
    annotation_img_1_path = os.path.join(ANNOTATIONS_FOLDERS_PATH, folder_1, 
                                         ('{}{}'.format(file_name_1, '.png')))
    annotation_img_1 = cv2.imread(annotation_img_1_path, cv2.IMREAD_GRAYSCALE)
    
    # Get contour
    contour_0 = data.contour
    contour_1_pred = contour_0.type(torch.DoubleTensor).add(translation_0_1_pred)
    
    # Create combined image
    _, combo_img_1, _, _ = compute_combo_img(contour_1_pred, osvos_img_1)
    combo_img_1_path = os.path.join(COMBO_RESULTS_FOLDERS_PATH, folder_1, 
                                    ('{}{}'.format(file_name_1, '.png')))
    
    if not os.path.exists(os.path.join(COMBO_RESULTS_FOLDERS_PATH, folder_1)):
        os.makedirs(os.path.join(COMBO_RESULTS_FOLDERS_PATH, folder_1))
        
    cv2.imwrite(combo_img_1_path, combo_img_1*255)
    
    #Compute J
    J_combo = db_eval_iou(annotation_img_1, combo_img_1)
    J_osvos = db_eval_iou(annotation_img_1, osvos_img_1_gray)
    
    #Compute F
    F_combo = db_eval_boundary(combo_img_1, annotation_img_1)
    F_osvos = db_eval_boundary(osvos_img_1_gray, annotation_img_1)
    
    # Compute metrics per sequence
    if i == 0:
        running_index = 0
        running_J_combo = 0.
        running_J_osvos = 0.
        running_F_combo = 0.
        running_F_osvos = 0.
        folder_old = folder_1

    # Update metrics if sequence is the same
    if folder_old == folder_1:
        running_index += 1
        running_J_combo += J_combo
        running_J_osvos += J_osvos
        running_F_combo += F_combo
        running_F_osvos += F_osvos
    # Calculate mean, print, and reset if new sequence
    else:
        mean_J_combo = running_J_combo / running_index
        mean_J_osvos = running_J_osvos / running_index
        mean_F_combo = running_F_combo / running_index
        mean_F_osvos = running_F_osvos / running_index
        
        print(folder_old)
        print('\tmean_J_combo: {}, mean_J_osvos: {}'.format(mean_J_combo, mean_J_osvos))
        print('\tmean_F_combo: {}, mean_F_osvos: {}'.format(mean_F_combo, mean_F_osvos))
        
        folder_old = folder_1
        running_index = 0
        running_J_combo = 0.
        running_J_osvos = 0.
        running_F_combo = 0.
        running_F_osvos = 0.
        

blackswan
	mean_J_combo: 0.9331513836169031, mean_J_osvos: 0.4927696365990751
	mean_F_combo: 0.9736308032309711, mean_F_osvos: 0.23274910348006242
bmx-trees
	mean_J_combo: 0.5678314884012534, mean_J_osvos: 0.13504030115904977
	mean_F_combo: 0.8219147803971817, mean_F_osvos: 0.09025482675937375
breakdance
	mean_J_combo: 0.6465887721948953, mean_J_osvos: 0.6329334438674871
	mean_F_combo: 0.6198345960244219, mean_F_osvos: 0.5233088966914324
camel
	mean_J_combo: 0.9362051453741372, mean_J_osvos: 0.6383907652464708
	mean_F_combo: 0.9789206725525882, mean_F_osvos: 0.5110143708977787
car-roundabout
	mean_J_combo: 0.9390719858509274, mean_J_osvos: 0.8192449854935715
	mean_F_combo: 0.8807811054996664, mean_F_osvos: 0.6026523402719486
car-shadow
	mean_J_combo: 0.9133176861529722, mean_J_osvos: 0.7602413654781216
	mean_F_combo: 0.9300617209438709, mean_F_osvos: 0.5333066001954953
cows
	mean_J_combo: 0.9429151799972249, mean_J_osvos: 0.6975684934921457
	mean_F_combo: 0.9769771927722277, mean_F_osv